## ایمبیڈنگز

پچھلی مثال میں، ہم نے `vocab_size` کی لمبائی والے ہائی ڈائمینشنل بیگ آف ورڈز ویکٹرز پر کام کیا تھا، اور ہم واضح طور پر لو ڈائمینشنل پوزیشنل ریپریزنٹیشن ویکٹرز کو اسپارس ون-ہاٹ ریپریزنٹیشن میں تبدیل کر رہے تھے۔ یہ ون-ہاٹ ریپریزنٹیشن میموری کے لحاظ سے مؤثر نہیں ہے، اس کے علاوہ، ہر لفظ کو ایک دوسرے سے آزاد سمجھا جاتا ہے، یعنی ون-ہاٹ انکوڈڈ ویکٹرز الفاظ کے درمیان کسی بھی معنوی مشابہت کو ظاہر نہیں کرتے۔

اس یونٹ میں، ہم **News AG** ڈیٹاسیٹ کو مزید دریافت کریں گے۔ شروع کرنے کے لیے، آئیے ڈیٹا لوڈ کرتے ہیں اور پچھلے نوٹ بک سے کچھ تعریفیں حاصل کرتے ہیں۔


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## ایمبیڈنگ کیا ہے؟

**ایمبیڈنگ** کا تصور یہ ہے کہ الفاظ کو کم جہتی گنجان ویکٹرز کے ذریعے ظاہر کیا جائے، جو کسی نہ کسی طرح ایک لفظ کے معنوی مطلب کو ظاہر کرتے ہیں۔ ہم بعد میں بات کریں گے کہ معنی خیز لفظ ایمبیڈنگ کیسے بنائی جائے، لیکن فی الحال ایمبیڈنگ کو ایک طریقہ سمجھیں جس سے لفظ ویکٹر کی جہتی تعداد کم کی جا سکتی ہے۔

ایمبیڈنگ لیئر ایک لفظ کو ان پٹ کے طور پر لے گی اور مخصوص `embedding_size` کے مطابق ایک آؤٹ پٹ ویکٹر پیدا کرے گی۔ ایک لحاظ سے، یہ `Linear` لیئر سے بہت مشابہ ہے، لیکن یہ ایک-ہاٹ انکوڈڈ ویکٹر لینے کے بجائے، لفظ نمبر کو ان پٹ کے طور پر لے سکے گی۔

اپنے نیٹ ورک میں ایمبیڈنگ لیئر کو پہلی لیئر کے طور پر استعمال کرتے ہوئے، ہم بیگ-آف-ورڈز ماڈل سے **ایمبیڈنگ بیگ** ماڈل میں تبدیل ہو سکتے ہیں، جہاں ہم پہلے اپنے متن کے ہر لفظ کو متعلقہ ایمبیڈنگ میں تبدیل کرتے ہیں، اور پھر ان تمام ایمبیڈنگز پر کوئی مجموعی فنکشن جیسے `sum`, `average` یا `max` کا حساب لگاتے ہیں۔

![پانچ سلسلہ وار الفاظ کے لیے ایمبیڈنگ کلاسیفائر دکھانے والی تصویر۔](../../../../../translated_images/ur/embedding-classifier-example.b77f021a7ee67eee.webp)

ہمارا کلاسیفائر نیورل نیٹ ورک ایمبیڈنگ لیئر سے شروع ہوگا، پھر ایگریگیشن لیئر، اور اس کے اوپر ایک لینیئر کلاسیفائر ہوگا:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### متغیر ترتیب کے سائز سے نمٹنا

اس معماری کے نتیجے میں، ہمارے نیٹ ورک کے لیے منی بیچز کو ایک خاص طریقے سے تیار کرنا ہوگا۔ پچھلے یونٹ میں، جب بیگ آف ورڈز استعمال کر رہے تھے، تو منی بیچ میں موجود تمام BoW ٹینسرز کا سائز `vocab_size` کے برابر ہوتا تھا، چاہے ہمارے متن کی ترتیب کی اصل لمبائی کچھ بھی ہو۔ جب ہم ورڈ ایمبیڈنگز کی طرف بڑھتے ہیں، تو ہر متن کے نمونے میں مختلف تعداد میں الفاظ ہوں گے، اور جب ان نمونوں کو منی بیچز میں یکجا کریں گے تو ہمیں کچھ پیڈنگ کا اطلاق کرنا ہوگا۔

یہ کام ڈیٹاسورس کو `collate_fn` فنکشن فراہم کرنے کی تکنیک کے ذریعے کیا جا سکتا ہے:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### ایمبیڈنگ کلاسیفائر کی تربیت

اب جب کہ ہم نے مناسب ڈیٹا لوڈر کی وضاحت کر دی ہے، ہم پچھلے یونٹ میں بیان کردہ تربیتی فنکشن کا استعمال کرتے ہوئے ماڈل کو تربیت دے سکتے ہیں:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **نوٹ**: ہم یہاں صرف 25,000 ریکارڈز کے لیے تربیت کر رہے ہیں (ایک مکمل ایپوک سے کم) وقت کی بچت کے لیے، لیکن آپ تربیت جاری رکھ سکتے ہیں، کئی ایپوکز کے لیے تربیت دینے کے لیے ایک فنکشن لکھ سکتے ہیں، اور زیادہ درستگی حاصل کرنے کے لیے لرننگ ریٹ پیرامیٹر کے ساتھ تجربہ کر سکتے ہیں۔ آپ تقریباً 90% درستگی تک پہنچنے کے قابل ہونا چاہیے۔


### ایمبیڈنگ بیگ لیئر اور مختلف لمبائی کی ترتیب کی نمائندگی

پچھلی آرکیٹیکچر میں، ہمیں تمام ترتیبوں کو ایک ہی لمبائی تک بڑھانا پڑتا تھا تاکہ انہیں ایک منی بیچ میں فٹ کیا جا سکے۔ یہ مختلف لمبائی کی ترتیبوں کو ظاہر کرنے کا سب سے مؤثر طریقہ نہیں ہے - ایک اور طریقہ یہ ہو سکتا ہے کہ **آفسیٹ** ویکٹر استعمال کیا جائے، جو ایک بڑے ویکٹر میں محفوظ تمام ترتیبوں کے آفسیٹس کو رکھے۔

![آفسیٹ ترتیب کی نمائندگی دکھانے والی تصویر](../../../../../translated_images/ur/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **نوٹ**: اوپر دی گئی تصویر میں، ہم حروف کی ترتیب دکھا رہے ہیں، لیکن ہمارے مثال میں ہم الفاظ کی ترتیب کے ساتھ کام کر رہے ہیں۔ تاہم، آفسیٹ ویکٹر کے ساتھ ترتیبوں کی نمائندگی کا عمومی اصول وہی رہتا ہے۔

آفسیٹ نمائندگی کے ساتھ کام کرنے کے لیے، ہم [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) لیئر استعمال کرتے ہیں۔ یہ `Embedding` کی طرح ہے، لیکن یہ مواد ویکٹر اور آفسیٹ ویکٹر کو ان پٹ کے طور پر لیتا ہے، اور اس میں اوسط لینے کی لیئر بھی شامل ہوتی ہے، جو `mean`، `sum` یا `max` ہو سکتی ہے۔

یہاں ایک ترمیم شدہ نیٹ ورک ہے جو `EmbeddingBag` استعمال کرتا ہے:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

ڈیٹاسیٹ کو تربیت کے لیے تیار کرنے کے لیے، ہمیں ایک تبدیلی فنکشن فراہم کرنا ہوگا جو آفسیٹ ویکٹر کو تیار کرے گا۔


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

نوٹ کریں کہ تمام پچھلی مثالوں کے برعکس، ہمارا نیٹ ورک اب دو پیرامیٹرز قبول کرتا ہے: ڈیٹا ویکٹر اور آفسیٹ ویکٹر، جو مختلف سائز کے ہیں۔ اسی طرح، ہمارا ڈیٹا لوڈر بھی ہمیں 2 کے بجائے 3 قدریں فراہم کرتا ہے: دونوں ٹیکسٹ اور آفسیٹ ویکٹرز کو خصوصیات کے طور پر فراہم کیا جاتا ہے۔ لہذا، ہمیں اپنی تربیتی فنکشن کو تھوڑا سا ایڈجسٹ کرنے کی ضرورت ہے تاکہ اس کا خیال رکھا جا سکے:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## سیمینٹک ایمبیڈنگز: ورڈ2ویک

پچھلی مثال میں، ماڈل کی ایمبیڈنگ لیئر نے الفاظ کو ویکٹر ریپریزنٹیشن میں میپ کرنا سیکھا، لیکن اس ریپریزنٹیشن میں زیادہ معنوی مطلب نہیں تھا۔ یہ بہتر ہوگا کہ ہم ایسا ویکٹر ریپریزنٹیشن سیکھیں، جہاں ملتے جلتے الفاظ یا مترادفات ایسے ویکٹرز کے مطابق ہوں جو کسی ویکٹر فاصلے (مثلاً یُوکلیڈین فاصلے) کے لحاظ سے ایک دوسرے کے قریب ہوں۔

ایسا کرنے کے لیے، ہمیں اپنے ایمبیڈنگ ماڈل کو ایک خاص طریقے سے بڑے ٹیکسٹ ڈیٹا پر پہلے سے ٹرین کرنا ہوگا۔ سیمینٹک ایمبیڈنگز کو ٹرین کرنے کے ابتدائی طریقوں میں سے ایک کو [ورڈ2ویک](https://en.wikipedia.org/wiki/Word2vec) کہا جاتا ہے۔ یہ دو بنیادی آرکیٹیکچرز پر مبنی ہے جو الفاظ کی ڈسٹریبیوٹڈ ریپریزنٹیشن پیدا کرنے کے لیے استعمال ہوتے ہیں:

- **کنٹینیوس بیگ آف ورڈز** (CBoW) — اس آرکیٹیکچر میں، ماڈل کو ارد گرد کے کانٹیکسٹ سے ایک لفظ کی پیش گوئی کرنے کے لیے ٹرین کیا جاتا ہے۔ دیے گئے این گرام $(W_{-2},W_{-1},W_0,W_1,W_2)$ میں، ماڈل کا مقصد $(W_{-2},W_{-1},W_1,W_2)$ سے $W_0$ کی پیش گوئی کرنا ہوتا ہے۔
- **کنٹینیوس اسکیپ-گرام** CBoW کے برعکس ہے۔ ماڈل موجودہ لفظ کی پیش گوئی کے لیے کانٹیکسٹ کے ارد گرد کے الفاظ کا استعمال کرتا ہے۔

CBoW تیز ہے، جبکہ اسکیپ-گرام سست ہے، لیکن کم استعمال ہونے والے الفاظ کی بہتر نمائندگی کرتا ہے۔

![CBoW اور اسکیپ-گرام الگورتھمز کو الفاظ کو ویکٹرز میں تبدیل کرنے کے لیے دکھانے والی تصویر۔](../../../../../translated_images/ur/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

گوگل نیوز ڈیٹاسیٹ پر پہلے سے ٹرین کیے گئے ورڈ2ویک ایمبیڈنگ کے ساتھ تجربہ کرنے کے لیے، ہم **gensim** لائبریری استعمال کر سکتے ہیں۔ نیچے ہم 'neural' کے سب سے زیادہ مشابہ الفاظ تلاش کرتے ہیں۔

> **نوٹ:** جب آپ پہلی بار ورڈ ویکٹرز بناتے ہیں، تو انہیں ڈاؤنلوڈ کرنے میں کچھ وقت لگ سکتا ہے!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ہم لفظ سے ویکٹر ایمبیڈنگز بھی حساب کر سکتے ہیں، جو تربیتی درجہ بندی ماڈل میں استعمال ہوں گے (وضاحت کے لیے ہم صرف ویکٹر کے پہلے 20 اجزاء دکھاتے ہیں):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

زبردست بات سیمانٹیکل ایمبیڈنگز کے بارے میں یہ ہے کہ آپ ویکٹر انکوڈنگ کو تبدیل کرکے معنیات کو بدل سکتے ہیں۔ مثال کے طور پر، ہم ایک ایسا لفظ تلاش کرنے کے لیے کہہ سکتے ہیں جس کی ویکٹر نمائندگی *بادشاہ* اور *عورت* کے الفاظ کے جتنا قریب ہو سکے، اور *مرد* کے لفظ سے جتنا دور ہو سکے:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

دونوں CBoW اور Skip-Grams "پیشگوئی کرنے والے" ایمبیڈنگز ہیں، کیونکہ یہ صرف مقامی سیاق و سباق کو مدنظر رکھتے ہیں۔ Word2Vec عالمی سیاق و سباق کا فائدہ نہیں اٹھاتا۔

**FastText**، Word2Vec پر مبنی ہے اور ہر لفظ کے لیے ویکٹر نمائندگی سیکھتا ہے، ساتھ ہی ہر لفظ میں موجود کردار n-grams کی بھی۔ ان نمائندگیوں کی قدریں ہر تربیتی مرحلے میں ایک ویکٹر میں اوسط کی جاتی ہیں۔ اگرچہ یہ پیشگی تربیت میں اضافی حساب کتاب شامل کرتا ہے، لیکن یہ لفظی ایمبیڈنگز کو ذیلی لفظی معلومات کو انکوڈ کرنے کے قابل بناتا ہے۔

ایک اور طریقہ، **GloVe**، ہم وقوع میٹرکس کے تصور کا فائدہ اٹھاتا ہے اور ہم وقوع میٹرکس کو زیادہ مؤثر اور غیر خطی لفظی ویکٹرز میں تقسیم کرنے کے لیے نیورل طریقے استعمال کرتا ہے۔

آپ مثال کے ساتھ کھیل سکتے ہیں اور ایمبیڈنگز کو FastText اور GloVe میں تبدیل کر سکتے ہیں، کیونکہ gensim کئی مختلف لفظی ایمبیڈنگ ماڈلز کو سپورٹ کرتا ہے۔


## پائی ٹارچ میں پہلے سے تربیت یافتہ ایمبیڈنگز کا استعمال

ہم اوپر دیے گئے مثال کو تبدیل کر سکتے ہیں تاکہ ایمبیڈنگ لیئر کے میٹرکس کو پہلے سے موجود معنوی ایمبیڈنگز، جیسے کہ Word2Vec، سے بھر سکیں۔ ہمیں یہ بات ذہن میں رکھنی ہوگی کہ پہلے سے تربیت یافتہ ایمبیڈنگز اور ہمارے ٹیکسٹ کارپس کے الفاظ کے ذخیرے ممکنہ طور پر ایک جیسے نہیں ہوں گے، اس لیے ہم ان الفاظ کے لیے وزن کو تصادفی اقدار سے شروع کریں گے جو موجود نہیں ہیں:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


اب آئیے اپنے ماڈل کو تربیت دیں۔ نوٹ کریں کہ ماڈل کو تربیت دینے میں لگنے والا وقت پچھلی مثال کے مقابلے میں نمایاں طور پر زیادہ ہے، کیونکہ ایمبیڈنگ لیئر کا سائز بڑا ہے، اور اس کے نتیجے میں پیرامیٹرز کی تعداد بہت زیادہ ہے۔ نیز، اسی وجہ سے، اگر ہم اوورفٹنگ سے بچنا چاہتے ہیں تو ہمیں اپنے ماڈل کو مزید مثالوں پر تربیت دینے کی ضرورت ہو سکتی ہے۔


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

ہمارے معاملے میں ہمیں درستگی میں بہت زیادہ اضافہ نظر نہیں آتا، جو ممکنہ طور پر مختلف الفاظ کے ذخیرے کی وجہ سے ہے۔  
مختلف الفاظ کے ذخیرے کے مسئلے کو حل کرنے کے لیے، ہم درج ذیل میں سے کسی ایک حل کا استعمال کر سکتے ہیں:  
* ہمارے الفاظ کے ذخیرے پر word2vec ماڈل کو دوبارہ تربیت دیں  
* پہلے سے تربیت یافتہ word2vec ماڈل کے ذخیرے کے ساتھ ہمارا ڈیٹا سیٹ لوڈ کریں۔ ڈیٹا سیٹ کو لوڈ کرنے کے لیے استعمال ہونے والے ذخیرے کو لوڈنگ کے دوران مخصوص کیا جا سکتا ہے۔  

دوسرا طریقہ زیادہ آسان لگتا ہے، خاص طور پر کیونکہ PyTorch `torchtext` فریم ورک میں ایمبیڈنگز کے لیے بلٹ ان سپورٹ موجود ہے۔ ہم، مثال کے طور پر، GloVe پر مبنی ذخیرہ درج ذیل طریقے سے بنا سکتے ہیں:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


لوڈ کردہ الفاظ میں درج ذیل بنیادی آپریشنز شامل ہیں:
* `vocab.stoi` ڈکشنری ہمیں کسی لفظ کو اس کے ڈکشنری انڈیکس میں تبدیل کرنے کی اجازت دیتی ہے
* `vocab.itos` اس کے برعکس کام کرتا ہے - نمبر کو لفظ میں تبدیل کرتا ہے
* `vocab.vectors` ایمبیڈنگ ویکٹرز کی صف ہے، لہذا کسی لفظ `s` کی ایمبیڈنگ حاصل کرنے کے لیے ہمیں `vocab.vectors[vocab.stoi[s]]` استعمال کرنا ہوگا

یہاں ایمبیڈنگز کے ساتھ چھیڑ چھاڑ کی ایک مثال دی گئی ہے تاکہ مساوات **kind-man+woman = queen** کو ظاہر کیا جا سکے (مجھے اسے کام کرنے کے لیے تھوڑا سا عددی گتانک ایڈجسٹ کرنا پڑا):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

ڈیٹاسیٹ کو GloVe کے ذخیرہ الفاظ کا استعمال کرتے ہوئے انکوڈ کرنے کے بعد، ہم ان ایمبیڈنگز کا استعمال کرتے ہوئے کلاسیفائر کو تربیت دے سکتے ہیں۔


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

جیسا کہ ہم نے اوپر دیکھا، تمام ویکٹر ایمبیڈنگز `vocab.vectors` میٹرکس میں محفوظ کی جاتی ہیں۔ یہ ان وزنوں کو ایمبیڈنگ لیئر کے وزنوں میں آسانی سے کاپی کرنے کے ذریعے لوڈ کرنا بہت آسان بناتا ہے۔


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

اب آئیے اپنے ماڈل کو تربیت دیں اور دیکھیں کہ کیا ہمیں بہتر نتائج ملتے ہیں:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

ایک وجہ جس کی وجہ سے ہم درستگی میں نمایاں اضافہ نہیں دیکھ رہے ہیں یہ ہے کہ ہمارے ڈیٹا سیٹ کے کچھ الفاظ پہلے سے تربیت یافتہ GloVe لغت میں موجود نہیں ہیں، اور اس طرح انہیں بنیادی طور پر نظر انداز کیا جاتا ہے۔ اس حقیقت پر قابو پانے کے لیے، ہم اپنے ڈیٹا سیٹ پر اپنی خود کی ایمبیڈنگز تربیت دے سکتے ہیں۔


## سیاقی ایمبیڈنگز

روایتی پری ٹرینڈ ایمبیڈنگز جیسے Word2Vec کی ایک اہم حد بندی یہ ہے کہ یہ الفاظ کے مختلف معنی کو واضح کرنے میں ناکام رہتی ہیں۔ اگرچہ پری ٹرینڈ ایمبیڈنگز کچھ حد تک الفاظ کے سیاق و سباق میں معنی کو سمجھ سکتی ہیں، لیکن ایک لفظ کے تمام ممکنہ معنی ایک ہی ایمبیڈنگ میں شامل کیے جاتے ہیں۔ یہ مسئلہ نیچے کے ماڈلز میں مشکلات پیدا کر سکتا ہے، کیونکہ بہت سے الفاظ جیسے 'play' کے مختلف معنی ہوتے ہیں جو اس سیاق و سباق پر منحصر ہوتے ہیں جس میں وہ استعمال کیے جاتے ہیں۔

مثال کے طور پر، لفظ 'play' ان دو مختلف جملوں میں بالکل مختلف معنی رکھتا ہے:
- میں تھیٹر میں ایک **play** دیکھنے گیا۔
- جان اپنے دوستوں کے ساتھ **play** کرنا چاہتا ہے۔

اوپر دی گئی پری ٹرینڈ ایمبیڈنگز لفظ 'play' کے دونوں معنی کو ایک ہی ایمبیڈنگ میں ظاہر کرتی ہیں۔ اس حد بندی کو دور کرنے کے لیے، ہمیں **زبان ماڈل** کی بنیاد پر ایمبیڈنگز بنانی ہوں گی، جو ایک بڑے متن کے مجموعے پر تربیت یافتہ ہوتا ہے اور *جانتا ہے* کہ الفاظ کو مختلف سیاق و سباق میں کیسے ترتیب دیا جا سکتا ہے۔ سیاقی ایمبیڈنگز پر تفصیل سے بات کرنا اس سبق کے دائرہ کار سے باہر ہے، لیکن ہم ان پر واپس آئیں گے جب اگلے یونٹ میں زبان ماڈلز پر بات کریں گے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
